In [ ]:
from keras import optimizers
from keras.layers.recurrent import LSTM
from keras.layers import Reshape, Lambda, BatchNormalization
import keras_preprocessing
from keras_preprocessing.image import ImageDataGenerator
from keras.applications.resnet_v2 import preprocess_input
import pandas as pd
import os
import keras
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.layers import Dense, Input, Activation
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint
import pickle
tf.debugging.set_log_device_placement(True)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

    except RuntimeError as e:
        print(e)

In [ ]:
input = Input(shape=(23, 267))

x = LSTM(128, return_sequences=True, kernel_initializer='he_normal', name='lstm1')(input)
x = LSTM(128, return_sequences=True, kernel_initializer='he_normal', name='lstm2')(x)
x = LSTM(128, kernel_initializer='he_normal', name='lstm3')(x)
x = Dense(2, kernel_initializer='he_normal',name='dense1')(x)
x = Activation('softmax', name='softmax')(x)
model = Model(input, x)

model.summary()

In [ ]:
with open('./ai_data/sampling_256_train.pickle','rb') as f:
    train_dataset=pickle.load(f)
with open('./ai_data/label_256_train.pickle','rb') as f:
    train_label=pickle.load(f)

with open('./ai_data/sampling_256_val.pickle','rb') as f:
    val_dataset=pickle.load(f)
with open('./ai_data/label_256_val.pickle','rb') as f:
    val_label=pickle.load(f)

In [ ]:
from typing import List, Tuple
from keras.callbacks import Callback
from sklearn.metrics import classification_report
import numpy as np
import itertools

class Metrics(Callback):

    def __init__(self, val_data,val_label,batch_size: int = 32):
        super().__init__()
        self.validation_data = val_data
        self.validation_label = val_label
        self.batch_size = batch_size

    def on_epoch_end(self, epoch, logs={}):
        y_preds = []
        y_trues = []
        data_gen=iter(self.validation_data)
        label_gen=iter(self.validation_label)
        for idx in range(len(self.validation_data)):
            X = next(data_gen)
            y_true=next(label_gen)
            y_trues.append([y_true])
            y_pred = self.model.predict([X])
            y_preds.append(y_pred)


        y_true = np.concatenate(y_trues, axis=0)
        y_pred = np.concatenate(y_preds, axis=0)
        val_true, val_pred = y_true,y_pred
        val_true_class = y_true
        val_pred_class = np.argmax(val_pred, axis=1)
        print(val_pred_class)
        cls_report = classification_report(
            y_true=val_true_class,
            y_pred=val_pred_class,
            output_dict=True,
            target_names=['0','1'],
            labels=np.arange(2)        )
        with open('./weights_multi/logs','a') as f:
            f.write(str(epoch)+'         '+str(cls_report)+'\n')
        print(cls_report)





In [ ]:
with tf.device('/gpu:0'):
    sgd = optimizers.SGD(lr=0.01, decay=1e-4, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='binary_crossentropy')
    filename = './weights_multi/weights-improvement-{epoch:02d}.hdf5'
    checkpoint = ModelCheckpoint(filename,             
                                 monitor='val_loss',   
                                 verbose=1,            
                                 save_best_only=False,  
                                 mode='auto'           
                                )
    history = model.fit(train_dataset['train'],train_label['train'],
                        epochs=10,batch_size=32,callbacks=[checkpoint,Metrics(val_dataset['train'], val_label['train'])])



In [ ]:
with open('./ai_data/sampling_256_test.pickle','rb') as f:
    test_dataset=pickle.load(f)
with open('./ai_data/label_256_test.pickle','rb') as f:
    test_label=pickle.load(f)

In [ ]:
from typing import List, Tuple
from keras.callbacks import Callback
from sklearn.metrics import classification_report
import numpy as np
import itertools

def test_Metrics(val_data,val_label,model):


        validation_data = val_data
        validation_label = val_label
        y_preds = []
        y_trues = []
        data_gen=iter(validation_data)
        label_gen=iter(validation_label)
        for idx in range(len(validation_data)):
            X = next(data_gen)
            y_true=next(label_gen)
            y_trues.append([y_true])
            y_pred = model.predict([X])
            y_preds.append(y_pred)


        y_true = np.concatenate(y_trues, axis=0)
        y_pred = np.concatenate(y_preds, axis=0)
        val_true, val_pred = y_true,y_pred
        val_true_class = y_true
        val_pred_class = np.argmax(val_pred, axis=1)
        print(val_pred_class)
        cls_report = classification_report(
            y_true=val_true_class,
            y_pred=val_pred_class,
            output_dict=True,
            target_names=['0','1'],
            labels=np.arange(2)        )
        with open('./weights_multi/logs','a') as f:
            f.write(str('result')+'         '+str(cls_report)+'\n')
        print(cls_report)

In [ ]:
from keras.models import load_model

model = load_model('./weights_multi/weights-improvement-05.hdf5')

test_Metrics(test_dataset['train'], test_label['train'],model)